Import and cleaning


In [1]:
!python -m pip install json-lines

In [2]:
import json_lines

with open('Illinois-20200302-xml/data/data.jsonl/data.jsonl', 'rb') as f: # opening file in binary(rb) mode 
    df = list(json_lines.reader(f))
    for item in json_lines.reader(f):
        print(item) #or use print(item['X']) for printing specific data
        break

In [3]:
import json

tot = len(df)
json_list = []

for n in range(0,tot):
    json_list.append(json.dumps(df[n]['casebody']))


In [4]:
import re
import nltk
import json
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

def clean_text(text):
    # Convert words to lower case
    text = text.lower() 

    text = re.sub('<.*?>', ' ', text)
    text = re.sub('[a-zA-Z]+\\\\[a-zA-Z]+', ' ', text)
    text = re.sub('\\\\[a-zA-Z]+', ' ', text)
    text = re.sub('\\\\"', '"', text)
    text = re.sub('[0-9]+[a-zA-Z]+', '', text)
    text = re.sub('\*', '', text)
    text = re.sub('amp', '', text)
    text = re.sub('[0-9]+', '', text)
    
    #Remove stopwords
    stop_words = set(stopwords.words('english'))
    text = " ".join([i for i in text.split() if i not in stop_words])
    
    #Remove punctuation
    words = nltk.word_tokenize(text)
    text = [word for word in words if word.isalnum()]

    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tompe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tompe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Topic Model

In [5]:
# for topic model

texts = []

for sentence in json_list:
    texts.append(clean_text(sentence))

In [6]:
# Necessary because pyLDAvis make some trouble

from IPython.display import HTML
css_str = '<style> \
.jp-icon-warn0 path {fill: var(--jp-warn-color0);} \
.bp3-button-text path { fill: var(--jp-inverse-layout-color3);} \
.jp-icon-brand0 path { fill: var(--jp-brand-color0);} \
text.terms { fill: #616161;} \
</style>'
display(HTML(css_str))

Topic model preparation

In [16]:
# Set training parameters.
num_topics = 2
chunksize = 500 #2000
passes = 50 #20
iterations = 600
eval_every = None  # Don't evaluate model perplexity, takes too much time.

C:\Users\tompe\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
# Remove rare and common tokens.
import gensim
from gensim.models import Word2Vec
from gensim.corpora import Dictionary
import gensim.corpora as corpora


C:\Users\tompe\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [9]:
# Create a dictionary representation of the documents.
id2word = corpora.Dictionary(texts)

corpus = [id2word.doc2bow(text) for text in texts]


LDA

In [32]:

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=chunksize,
                                           passes=passes,
                                           alpha='auto',
                                           per_word_topics=True,
                                           eval_every=eval_every,
                                           iterations=iterations)

In [18]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# Print the Keyword in the 10 topics
#pprint(lda_model.print_topics())
#doc_lda = lda_model[corpus]

lda_topics = lda_model.get_topics()

for topicno in range(num_topics):
    print('Topic {}'.format(topicno))
    print([(x, round(y, 2)) for x, y in lda_model.show_topic(topicno, topn=10)], '\n')

Topic 0
[('defendant', 0.02), ('court', 0.02), ('trial', 0.01), ('evidence', 0.01), ('app', 0.01), ('state', 0.01), ('case', 0.01), ('people', 0.0), ('would', 0.0), ('plaintiff', 0.0)] 

Topic 1
[('court', 0.01), ('said', 0.01), ('upon', 0.01), ('property', 0.01), ('judgment', 0.01), ('case', 0.0), ('plaintiff', 0.0), ('made', 0.0), ('act', 0.0), ('may', 0.0)] 



C:\Users\tompe\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [19]:
tid = id2word.token2id['court']
lda_model.get_term_topics(tid)

[(0, 0.01597645), (1, 0.013337535)]

In [13]:
!pip install pyLDAvis

In [20]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

import matplotlib.pyplot as plt
%matplotlib inline

# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x    y  topics  cluster      Freq
topic                                          
1      0.108423  0.0       1        1  52.30563
0     -0.108423  0.0       2        1  47.69437, topic_info=           Term          Freq         Total Category  logprob  loglift
1385  defendant  1.719665e+06  1.719665e+06  Default  30.0000  30.0000
1116      trial  1.130934e+06  1.130934e+06  Default  29.0000  29.0000
856    property  5.664510e+05  5.664510e+05  Default  28.0000  28.0000
981        said  8.242970e+05  8.242970e+05  Default  27.0000  27.0000
3178   contract  3.473740e+05  3.473740e+05  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
536    illinois  2.408295e+05  4.656207e+05   Topic2  -5.9025   0.0811
1105       time  2.623337e+05  6.045542e+05   Topic2  -5.8170  -0.0945
991     section  2.551258e+05  6.304861e+05   Topic2  -5.8448  -0.1644
755         one  2.367957e+05  5.637455e+05   Topic2  -5.9194  -0.1270
677         may  2.310809e+05  6.329241e+05   Topic2  -5.9438  -0.2672

[187 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
0         1  0.124298         a
0         2  0.875702         a
1992      1  0.028225  accident
1992      2  0.971781  accident
6430      1  0.999998     acres
...     ...       ...       ...
1172      2  0.723458      west
1176      1  0.348571   whether
1176      2  0.651430   whether
1188      1  0.474005     would
1188      2  0.525996     would

[248 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1])

In [21]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.31933423756280216


Latent Semantic Indexin

In [15]:
from gensim.models import LsiModel

model = LsiModel(corpus, id2word=id2word, num_topics=num_topics)

vectors = model[corpus]

type(vectors)

C:\Users\tompe\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


gensim.interfaces.TransformedCorpus

In [16]:
topics = model.get_topics()
topics.shape

C:\Users\tompe\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(10, 386575)

In [17]:

for topicno in range(num_topics):
    print('Topic {}'.format(topicno))
    print([(x, round(y, 2)) for x, y in model.show_topic(topicno, topn=10)], '\n')

Topic 0
[('court', 0.44), ('defendant', 0.4), ('trial', 0.22), ('evidence', 0.16), ('case', 0.15), ('would', 0.13), ('plaintiff', 0.13), ('said', 0.12), ('state', 0.12), ('people', 0.11)] 

Topic 1
[('defendant', 0.66), ('said', -0.21), ('trial', 0.16), ('people', 0.15), ('court', -0.14), ('upon', -0.13), ('property', -0.13), ('act', -0.12), ('plaintiff', -0.11), ('company', -0.1)] 

Topic 2
[('said', -0.51), ('court', 0.37), ('defendant', -0.25), ('trial', 0.2), ('company', -0.15), ('upon', -0.14), ('app', 0.14), ('section', 0.14), ('made', -0.12), ('motion', 0.11)] 

Topic 3
[('plaintiff', -0.75), ('court', 0.19), ('state', 0.17), ('people', 0.15), ('evidence', -0.14), ('section', 0.11), ('said', 0.11), ('county', 0.11), ('act', 0.1), ('jury', -0.1)] 

Topic 4
[('court', -0.42), ('act', 0.31), ('section', 0.24), ('trial', -0.2), ('state', 0.2), ('said', -0.19), ('city', 0.17), ('board', 0.16), ('public', 0.15), ('judgment', -0.12)] 

Topic 5
[('defendant', 0.46), ('evidence', -0.37),

C:\Users\tompe\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [49]:

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=model, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

C:\Users\tompe\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Coherence Score:  0.3867220104224676


In [22]:
tid = id2word.token2id['court']
lda_model.get_term_topics(tid)

[(0, 0.01597645), (1, 0.013337535)]

In [23]:
narcotics = ['cannabis', 'cocaine', 'methamphetamine', 'smart drugs', 'marijuana', 'MDMA', 'LSD', 'KETAMINA', 'heroin', 'fentanyl']
weapons = ['gun', 'knife', 'weapon', 'firearm', 'rifle', 'carabine', 'shotgun', 'assaults rifle', 'sword', 'blunt objects'] 
investigation = ['gang', 'mafia', 'serial kiler', 'rape', 'thefts', 'recidivism', 'arrest', 'ethnicity', 'caucasian', 'afroamerican', 'native american', 'hispanic', 'gender', 'male', 'female', 'man', 'woman', 'girl', 'boy', 'robbery', 'cybercrime']

In [33]:
for n in narcotics:
    try:
        tid = id2word.token2id[n]
        print(n + ":")
        print(lda_model.get_term_topics(tid))
    except:
        print(n + " not find")

cannabis:
[]
cocaine:
[]
methamphetamine not find
smart drugs not find
marijuana:
[]
MDMA not find
LSD not find
KETAMINA not find
heroin:
[]
fentanyl:
[]


In [34]:
for w in weapons:
    try:
        tid = id2word.token2id[w]
        print(w + ":")
        print(lda_model.get_term_topics(tid))
    except:
        print(w + " not find")

gun:
[]
knife:
[]
weapon:
[]
firearm:
[]
rifle:
[]
carabine not find
shotgun:
[]
assaults rifle not find
sword:
[]
blunt objects not find


In [35]:
for i in investigation:
    try:
        tid = id2word.token2id[i]
        print(i + ":")
        print(lda_model.get_term_topics(tid))
    except:
        print(i + " not find")

gang:
[]
mafia:
[]
serial kiler not find
rape:
[]
thefts:
[]
recidivism:
[]
arrest:
[]
ethnicity:
[]
caucasian:
[]
afroamerican not find
native american not find
hispanic:
[]
gender:
[]
male:
[]
female:
[]
man:
[]
woman:
[]
girl:
[]
boy:
[]
robbery:
[]
cybercrime not find
